In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, DoubleType

schema = StructType([ \
    StructField("num",IntegerType(),False), \
    StructField("date",DateType(),False), \
    StructField("resource_id",IntegerType(),False), \
    StructField("earnings", DoubleType(), False)
  ])

In [ ]:
data = (spark
                .read
                .csv("/data/D11.07.04G07.01.10.58.01.csv", header=True, schema=schema)
              )

data.printSchema()
data.show(5)

In [ ]:
earnings = data.groupBy("date").sum("earnings") \
                .withColumnRenamed("sum(earnings)", "total_earnings") \
                .withColumnRenamed("date", "earnings_date") \
                .orderBy(data.date.desc())

print(earnings.count())
earnings.show(5)

In [ ]:
dbtable = "Earnings"

#uses synapse MSI based auth
url = TokenLibrary.getSecret("kehsihbaazkeyvault1", "azsqldb-connection-string", "azkv")

try:
  earnings.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("append") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .save()
except ValueError as error :
    print("Connector write failed", error)

In [ ]:
dbtable = "Earnings"

#uses synapse MSI based auth
url = TokenLibrary.getSecret("kehsihbaazkeyvault1", "azsqldb-connection-string", "azkv")

data = spark.read \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .load()

print(data.count())
data.show()